### 支持三种分词模式：
- 精确模式，试图将句子最精确地切开，适合文本分析
- 全模式，把句子中所有的可以成词的词语都扫描出来，速度非常快，但不能解决歧义
- 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适用于搜索引擎分词
- 支持繁体分词
- 支持自定义词典

#### 分词 jieba.cut
- 第一个参数为需要分词的字符串
- cut_all参数用来控制是否采用全模式
- jieba.cut_for_search方法接受一个参数：需要分词的字符串，
    - 该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细
   
- jieba.cut以及jieba.cut_for_search返回的结构都是可迭代的generator，可以使用for循环来获得分词后得到的每一个词语，也可以用list(jieba.cut(..))转化为list

In [4]:
#encoding=utf-8
import jieba

seg_list = jieba.cut("我来到北京清华大学",cut_all=True)
print ("Full Mode:", "/ ".join(seg_list)) #全模式

seg_list = jieba.cut("我来到北京清华大学",cut_all=False)
print ("Default Mode:", "/ ".join(seg_list)) #精确模式

seg_list = jieba.cut("他来到了网易杭研大厦") #默认是精确模式
print (", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造") #搜索引擎模式
print (", ".join(seg_list))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 2.866 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


#
- 对比可以看出全模式情况下，它会找出所有可以组成词的划分，而精确模式与其对比给出的答案就是只有一个
- 关于HMM（hidden markov model:隐马尔可夫模型）参数，对于未登录词，采用了基于汉字成词能力的HMM模型，使用了Viterbi算法。

In [7]:
import jieba
seg_list = jieba.cut('他来到了网易杭研大厦',HMM=False)
print('HMM为false:'+'/'.join(seg_list))
seg_list = jieba.cut('他来到了网易杭研大厦',HMM=True)
print('HMM为True:'+'/'.join(seg_list))

HMM为false:他/来到/了/网易/杭/研/大厦
HMM为True:他/来到/了/网易/杭研/大厦


    - 所以一般情况下，使用cut方法，不用考虑HMM这个参数就可以，让它默认为True即可，让Viterbi算法为我们识别新词。HMM也能有效的解决中文中的歧义问题。

### 可以自己添加自定义字典
- 开发者可以指定自己自定义的词典，以便包含jieba词库里没有的词。虽然jieba有新词识别能力，但是自行添加新词可以保证更高的正确率
- 用法： jieba.load_userdict(file_name) # file_name为自定义词典的路径
- 词典格式和dict.txt一样，一个词占一行；每一行分三部分，一部分为词语，另一部分为词频，最后为词性（可省略），用空格隔开

### 词性标注

In [5]:
import jieba.posseg as pseg
words =pseg.cut("我爱北京天安门")
for w in words:
    print (w.word,w.flag)

我 r
爱 v
北京 ns
天安门 ns


### 并行分词
- jieba.enable_parallel(4) # 开启并行分词模式，参数为并行进程数jieba.disable_parallel() # 关闭并行分词模式

### Tokenize
- 返回词语在原文的起始位置，注意参数只接受unicode

In [6]:
result = jieba.tokenize(u'永和服装饰品有限公司')
for tk in result:
    print ("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

word 永和		 start: 0 		 end:2
word 服装		 start: 2 		 end:4
word 饰品		 start: 4 		 end:6
word 有限公司		 start: 6 		 end:10


### 获得前20词的权重大小 
- 用jieba.analyse.extract_tags

In [10]:
import jieba
import jieba.analyse
file_name = '1.txt'
content = open(file_name, 'rb').read()
tags = jieba.analyse.extract_tags(content, topK=20,withWeight=True)
for i in tags :
    print(i)

('鸿渐', 0.21638643763446852)
('辛楣', 0.12385522874320122)
('小姐', 0.06859330337911916)
('孙小姐', 0.06213265686414787)
('柔嘉', 0.054676889357323594)
('方鸿渐', 0.0496264917902045)
('自己', 0.033501413977364065)
('李梅亭', 0.028715200885500877)
('唐小姐', 0.028197268553720612)
('没有', 0.02806421719398845)
('知道', 0.023341341308918614)
('什么', 0.021301770841217118)
('高松年', 0.021241381476945854)
('先生', 0.02096255354677887)
('太太', 0.020325655465768554)
('汪太太', 0.019338798873938436)
('女人', 0.019230220369696297)
('可是', 0.019107288511088463)
('他们', 0.01772965595927792)
('赵辛楣', 0.017701151230788213)


##
- 去除某些词汇

In [15]:
import sys
import jieba
from os import path

d = path.dirname('__file__') # 获取当前文件的dir路径
stopwords_path = 'stopwords1893.txt'  # 停用词表路径

text_path = '1.txt' #《围城》的文本路径
text = open(path.join(d, text_path),'rb').read()

def RmStopWords(text):
    mywordlist = []
    seg_list = jieba.cut(text, cut_all=False)
    liststr="/ ".join(seg_list) # 添加切分符
    f_stop = open(stopwords_path,encoding='utf-8')
    try:
        f_stop_text = f_stop.read()
    finally:
        f_stop.close( )
    f_stop_seg_list=f_stop_text.split('\n') # 停用词是每行一个，所以用/n分离
    for myword in liststr.split('/'):
        #对于每个切分的词都去停用词表中对比
        if not(myword.strip() in f_stop_seg_list) and len(myword.strip())>1:
            mywordlist.append(myword)
    return ''.join(mywordlist) #返回一个字符串

txt2 = RmStopWords(text)
text_write = '2.txt'
with open(text_write,'w') as f:
    f.write(txt2)
    print("Success")

Success


In [16]:
import jieba
import jieba.analyse
file_name = '2.txt'
content = open(file_name, 'rb').read()
tags = jieba.analyse.extract_tags(content, topK=20,withWeight=True)#这句是关键
for i in tags :
    print(i)

('鸿渐', 0.28044557109396573)
('辛楣', 0.16052138358382909)
('小姐', 0.08889969422147594)
('孙小姐', 0.0805264351515704)
('柔嘉', 0.0708634590461673)
('方鸿渐', 0.06431793962523721)
('李梅亭', 0.03721606122568498)
('唐小姐', 0.036544799985091365)
('高松年', 0.027529689125849164)
('太太', 0.026342871195032088)
('汪太太', 0.02506386515607156)
('女人', 0.024923143025037845)
('赵辛楣', 0.022941407604874304)
('学生', 0.022611225297624257)
('李先生', 0.018797497809552442)
('结婚', 0.017076181219597433)
('明天', 0.016696820078620864)
('东西', 0.016488028327530227)
('仿佛', 0.014938565422774473)
('也许', 0.014352840250177188)
